<a href="https://colab.research.google.com/github/larsondg2000/Colab-Projects/blob/main/fine_tune_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install trl

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("databricks/databricks-dolly-15k")

In [ ]:
dataset_split = dataset["train"].train_test_split(test_size=.01)

In [ ]:
print(dataset_split)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 14860
    })
    test: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 151
    })
})


In [ ]:
# logits: [sequence_length, vocab_size]
# probability distrobution of where each loacation in the sequence length is
# predicting the next token over all possible tokens in the model vocab.

# Softmax give prob. distro.

import torch
import evaluate

def preprocess_logits_for_metrics(logits, labels):
  if isinstance(logits, tuple):
    logits=logits[0]
  return logits.argmax(dim=-1)


metric = evaluate.load('accuracy')

def compute_metrics(eval_preds):
  preds, labels = eval_preds

 # shift the labels and predictions
  labels = labels[:, 1:].reshape(-1)
  preds = preds[:, :-1].reshape(-1)

  mask = labels != -100 # boolean mask

  # apply the mask to filter out ignored labels and corresponding prdictions
  filtered_labels = labels[mask]
  filtered_preds = preds[mask]

  return metric.compute(predictions=filtered_preds,references=filtered_labels)


In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")
tokenizer.pad_token = tokenizer.eos_token # end of sequence token for padding


model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T", torch_dtype=torch.bfloat16, device_map='auto')

# processing a raw example
def formatting_prompts_func(example):
  output_texts = []
  for i in range(len(example['instruction'])):
    text = f"### Input: {example['context'][i] + ' ' + example['instruction'][i]}\n ### Output: {example['response'][i]}"
    output_texts.append(text)
  return output_texts

# add contect to response template
response_template_with_context = "### Input: Dummy Input\n ### Output:"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)
print(response_template_ids)

response_template_ids = response_template_ids[-1*len(tokenizer.encode(" ### Output:", add_special_tokens=False)):]
print(response_template_ids)

collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

[835, 10567, 29901, 360, 11770, 10567, 13, 835, 10604, 29901]
[13, 835, 10604, 29901]


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,

    gradient_accumulation_steps=32,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=10,
    weight_decay=0.01,
    evaluation_strategy='steps',
    eval_steps=10, #eval every 10 steps
    logging_steps=1,
    gradient_checkpointing=True, # recomputes forward pass activations in backward pass to save nmemory
    save_steps=500 # checkpoint every 500 steps

)

trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=dataset_split['train'],
    eval_dataset=dataset_split['test'],
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    max_seq_length=2048  # truncate sequences greater than 2048 tokens.  Increasing equals more memory useage

)
trainer.train()

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,1.491200,nan,0.597245
20,1.479600,nan,0.600456
30,1.603900,nan,0.595470
40,1.549600,nan,0.595977
50,1.108500,nan,0.598935
60,1.138800,nan,0.594455
70,1.695600,nan,0.597667
80,1.578300,nan,0.596737
90,1.720700,nan,0.596230
100,1.829000,nan,0.598090


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key `[13, 835, 10604, 29901]` in the following instance: <s> ### Input: The story begins only days after the conclusion of Taran Wanderer. With winter approaching, Taran and his companion Gurgi return from their wanderings to Caer Dallben after getting news from Kaw the crow that Princess Eilonwy has returned from the Isle of Mona. Indeed, they find her at home, along with her escort King Rhun of Mona and the former giant Glew, who had been magically restored to human size by a potion from Dallben.

Before Taran can propose to Eilonwy, the bard-king Fflewddur Fflam and his mount Llyan arrive with a gravely injured Gwydion, Prince of Don. Servants of Arawn had assaulted them and seized the magical black sword Dyrnwyn. Fflewddur also states that Taran was involved in the ambush, baffling everyone. With Achren's help, the truth is determined: Arawn himself has come from Annuvin to the ve